In [109]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
import os
from langchain.document_loaders import DirectoryLoader
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import openai

In [110]:
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(texts[1]) 

page_content='Iron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.' metadata={'source': 'new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}


In [111]:
print(len(texts))

3


In [112]:
metadata_list = [doc.metadata for doc in texts]
text_content_list = [doc.page_content for doc in texts]

In [113]:
print(type(text_content_list[1]))

<class 'str'>


In [114]:
def generate_ids(i):
    ids = []
    for a in range(0, i):
        id = "doc" + str(a) 
        ids.append(id)
    return ids

id_list=generate_ids(len(texts))

In [115]:
# from gradio_client import Client
# client = Client("https://tonic-e5.hf.space/--replicas/h4q86/")
# result = client.predict(
# 		"ArguAna",	# Literal['ArguAna', 'ClimateFEVER', 'DBPedia', 'FEVER', 'FiQA2018', 'HotpotQA', 'MSMARCO', 'NFCorpus', 'NQ', 'QuoraRetrieval', 'SCIDOCS', 'SciFact', 'Touche2020', 'TRECCOVID']  in 'Select a Task' Dropdown component
# 		text_content_list[1],	# str  in '📖Input Text' Textbox component
# 		api_name="/generate_and_format_embeddings"
# )
# print(result)

In [116]:
print(len(result['data'][0]['embedding'][0]))

4096


In [117]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class E5EmbeddingFunction(EmbeddingFunction):
    def __init__(self, api_link: str, model_name: str):
        if not api_link:
            raise ValueError("Please provide a api end point.")

        if not model_name:
            raise ValueError("Please provide the model name.")
        self._api_link = api_link
        self._model_name = model_name

    
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = []
        for text in input:
            try:
                from gradio_client import Client
                client = Client(self._api_link)
                
                response = client.predict(
                    self._model_name,
                    text,  
                    api_name="/generate_and_format_embeddings"
                )
                
                # Extracting the embeddings from the response
                embeddings_data = response.get('data', [])
                if embeddings_data:
                    embedding_list = embeddings_data[0].get('embedding', None)
                    print("Embeddings:", embedding_list)

                    if embedding_list is None:
                        print("Warning: Gradio client returned None.")
                        return None  

                    # Extending the embeddings list with the nested embeddings
                    embeddings.extend(embedding_list)
                else:
                    print("Warning: 'data' not found in response.")
                    return None
            except Exception as e:
                print(f"Error in E5EmbeddingFunction: {e}")
                return None

        return embeddings


In [129]:
embedding_function= embedding_functions.OpenAIEmbeddingFunction(
                api_key="None",
                api_base="https://tonic-e5.hf.space/--replicas/ax7dg/generate_and_format_embeddings",
                api_type="None",
                api_version="123",
                model_name="ClimateFEVER"
            )

In [127]:
import chromadb.utils.embedding_functions as embedding_functions
embedding_function = EmbeddingFunction.E5EmbeddingFunction(api_link="https://tonic-e5.hf.space/--replicas/ax7dg/", model="DBPedia")

AttributeError: type object 'EmbeddingFunction' has no attribute 'E5EmbeddingFunction'

In [130]:
collection_name = "texts"
existing_collections = [collection.name for collection in chroma_client.list_collections()]

if collection_name in existing_collections:
    chroma_client.delete_collection(collection_name)
    print(f"Info: Existing collection '{collection_name}' deleted.")
else:
    print("No collection found")

No collection found


In [131]:
vector_store = chroma_client.get_or_create_collection(name="texts",
                                                      embedding_function=embedding_function)
vector_store.add(ids=id_list, documents=text_content_list,metadatas=metadata_list)

NotFoundError: Error code: 404 - {'detail': 'Not Found'}

In [95]:
results = vector_store.query(
    query_texts="Who is Nitin Jayakrishnan",
    n_results=2
)
print(results)


Loaded as API: https://tonic-e5.hf.space/--replicas/ax7dg/ ✔
Embeddings: [[0.01154327392578125, 0.0170135498046875, -0.007244110107421875, 0.005615234375, 0.006069183349609375, -0.025238037109375, -0.0026092529296875, 0.03656005859375, 0.019561767578125, -0.01270294189453125, -0.0009469985961914062, 0.0004642009735107422, 0.0227508544921875, -0.0011005401611328125, 0.01023101806640625, -0.028900146484375, 0.00013828277587890625, 0.010498046875, 0.00547027587890625, -0.02496337890625, -0.0017957687377929688, -0.03411865234375, -0.001293182373046875, 0.0114288330078125, 0.0219573974609375, 0.0114593505859375, 0.01100921630859375, -0.00981903076171875, 0.0177001953125, 0.0156707763671875, -0.0083160400390625, -0.002964019775390625, 0.0062713623046875, 0.004512786865234375, 0.015960693359375, -0.0279541015625, -0.0147552490234375, -0.01058197021484375, 0.0147552490234375, -0.004665374755859375, -0.01277923583984375, -0.002964019775390625, -0.0169830322265625, -0.00264739990234375, 0.013252

In [99]:
load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")


In [104]:
query="Who is Nitin Jayakrishnan"
openai.api_key = OPENAI_API_KEY
completion = openai.chat.completions.create(
model="gpt-3.5-turbo",
temperature=0,
messages=
[
    {"role": "user",
    "content": 
    f"We have provided context information below. \n"
    f"---------------------\n"
    f"{results}"
    f"\n---------------------\n"
    f"Given this information, please answer the question: {query}"
    }
]
).choices[0].message.content
print(completion)


Nitin Jayakrishnan is the CEO and founder of Pando, an AI-powered supply chain startup.
